In [ ]:
from dotenv import load_dotenv

import requests
import tempfile

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.chat_models import ChatOpenAI  # ← Chat 모델 사용
from langchain.chains import StuffDocumentsChain, LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.retrieval_qa.base import RetrievalQA
from langchain.memory import ConversationBufferMemory
from langchain.storage import InMemoryStore

# (1) 문서 로드 (gist에서 다운로드 후 임시 저장)
url = "https://gist.githubusercontent.com/serranoarevalo/5acf755c2b8d83f1707ef266b82ea223/raw"
response = requests.get(url)
with tempfile.NamedTemporaryFile(delete=False, suffix=".txt", mode='w', encoding='utf-8') as f:
    f.write(response.text)
    temp_file_path = f.name

loader = TextLoader(temp_file_path)
docs = loader.load()

# (2) 문서 쪼개기
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = splitter.split_documents(docs)

# (3) 임베딩 생성 및 캐시 적용
embedding = OpenAIEmbeddings()
store = InMemoryStore()
cached_embedder = CacheBackedEmbeddings.from_bytes_store(embedding, store)

# (4) 벡터스토어 생성
vectorstore = FAISS.from_documents(split_docs, cached_embedder)
retriever = vectorstore.as_retriever()

# (5) 대화 메모리 설정
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# (6) 체인 수동 구성 (stuff chain)
prompt_template = """
당신은 질문에 정확하게 답하는 도우미입니다.
다음 문서 내용을 바탕으로 질문에 답해주세요.
모르겠으면 모른다고 하세요. 지어내지 마세요.

{context}

질문: {question}
답변:"""

prompt = PromptTemplate(
    input_variables=["context", "question"],
    template=prompt_template,
)

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")

llm_chain = LLMChain(llm=llm, prompt=prompt)
stuff_chain = StuffDocumentsChain(
    llm_chain=llm_chain,
    document_variable_name="context"
)

# (7) Retrieval QA 체인 구성
rag_chain = RetrievalQA(
    retriever=retriever,
    combine_documents_chain=stuff_chain,
    memory=memory,
    return_source_documents=False,
)

# (8) 질문 실행
questions = [
    "Is Aaronson guilty?",
    "What message did he write in the table?",
    "Who is Julia?"
]

for question in questions:
    print(f"\nQ: {question}")
    print("A:", rag_chain.run(question))

Created a chunk of size 1444, which is longer than the specified 1000
Created a chunk of size 1251, which is longer than the specified 1000
Created a chunk of size 1012, which is longer than the specified 1000
Created a chunk of size 2313, which is longer than the specified 1000
Created a chunk of size 1458, which is longer than the specified 1000
Created a chunk of size 1673, which is longer than the specified 1000
Created a chunk of size 1137, which is longer than the specified 1000
Created a chunk of size 1559, which is longer than the specified 1000
Created a chunk of size 1200, which is longer than the specified 1000
Created a chunk of size 1042, which is longer than the specified 1000
Created a chunk of size 1345, which is longer than the specified 1000
Created a chunk of size 1339, which is longer than the specified 1000
Created a chunk of size 1288, which is longer than the specified 1000
Created a chunk of size 1014, which is longer than the specified 1000
Created a chunk of s


Q: Is Aaronson guilty?
A: Aaronson is not guilty of betraying Julia, as confirmed by O'Brien in the passage.

Q: What message did he write in the table?
A: He wrote "GOD IS POWER" on the table.

Q: Who is Julia?
A: Julia is a character who is important to the protagonist and is still alive and in need of help according to the passage.
